# Testing with R

In [1]:
install.packages("sparklyr")

Installing package into ‘/usr/local/spark-3.0.1-bin-hadoop3.2/R/lib’
(as ‘lib’ is unspecified)



In [9]:
install.packages("tidyverse")

Installing package into ‘/usr/local/spark-3.0.1-bin-hadoop3.2/R/lib’
(as ‘lib’ is unspecified)

also installing the dependencies ‘highr’, ‘markdown’, ‘rematch’, ‘knitr’, ‘tinytex’, ‘xfun’, ‘clipr’, ‘BH’, ‘cpp11’, ‘cellranger’, ‘progress’, ‘fs’, ‘rmarkdown’, ‘whisker’, ‘selectr’, ‘broom’, ‘forcats’, ‘haven’, ‘hms’, ‘lubridate’, ‘modelr’, ‘readr’, ‘readxl’, ‘reprex’, ‘rvest’




### Import Libraries

In [10]:
library(sparklyr)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ purrr::invoke() masks sparklyr::invoke()
✖ dplyr::lag()    masks stats::lag()



In [3]:
conf <- spark_config()

In [4]:
conf$`sparklyr.cores.local` <- 6
conf$`sparklyr.shell.driver-memory` <- "12G"

In [5]:
sc <- spark_connect(master = "local", config = conf)

### Load Parquet Frames

In [6]:
system("ls data/*", intern=TRUE)

[1] "data/s3_parquet_logs.c000"

In [12]:
txt <- spark_read_parquet(sc, "logdata", "data/*")

### Build UDFs

In [74]:
test <- "there/are/a/lot/of/words"
dplyr::last(str_split(test, '/', simplify=TRUE))

[1] "words"

In [77]:
get_object <- function(string) {
    splitted <- str_split(string, '/', simplify=TRUE)
    s3_object <- dplyr::last(splitted)
    return(s3_object)
}

In [78]:
get_object(test)

[1] "words"

In [88]:
splitted <- str_split(test, '/', simplify=TRUE)
paste(splitted[,-ncol(splitted)], collapse="/")

[1] "there/are/a/lot/of"

In [96]:
get_prefix <- function(string) {
    splitted <- str_split(string, '/', simplify=TRUE)
    all_but_last <- splitted[,-ncol(splitted)]
    concat <- paste(all_but_last, collapse = "/")
    return(concat)
}

In [97]:
get_prefix(test)

[1] "there/are/a/lot/of"

### Process Dataframes

In [15]:
txt2 <- txt %>%
            filter(instr(key, "dmp")>0)

In [ ]:
txt2 %>%
    spark_apply(get_prefix, barrier = TRUE, columns = c(prefix = "string")) %>%
    collect()

In [ ]:
txt2 %>% 
    select(key, requestid, operation)